In [ ]:

import os

try: 
    import torch_tensorrt
    
except:
    !pip install /kaggle/input/pytorch112-cu113/{torch-1.12.1+cu113-cp37-cp37m-linux_x86_64.whl,torchvision-0.13.1+cu113-cp37-cp37m-linux_x86_64.whl}
    !pip install /kaggle/input/torch-tensorrt-pkg/nvidia_pyindex-1.0.9-py3-none-any.whl
    !mkdir -p /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cublas-cu11-2022.4.8.xyz /tmp/pip/cache/nvidia-cublas-cu11-2022.4.8.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cuda-runtime-cu11-2022.4.25.xyz /tmp/pip/cache/nvidia-cuda-runtime-cu11-2022.4.25.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia-cudnn-cu11-2022.5.19.xyz /tmp/pip/cache/nvidia-cudnn-cu11-2022.5.19.tar.gz
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cublas_cu117-11.10.1.25-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cuda_runtime_cu117-11.7.60-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_cudnn_cu116-8.4.0.27-py3-none-manylinux1_x86_64.whl /tmp/pip/cache/
    !cp /kaggle/input/torch-tensorrt-pkg/nvidia_tensorrt-8.4.3.1-cp37-none-linux_x86_64.whl /tmp/pip/cache/
    !pip install --no-index --find-links /tmp/pip/cache/ nvidia_tensorrt
    
    #install torch_tensorrt
    #!pip install /kaggle/input/torch-tensorrt-pkg/torch_tensorrt-1.2.0-cp37-cp37m-linux_x86_64.whl
    

try: 
    import dicomsdl
    
except:
    !pip install /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
    !pip install /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
    !pip install /kaggle/input/nvidia-dali-wheel/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
    !cp /kaggle/input/easy-load-the-image-with-nvjpeg2000/nvjpeg2k.so ./
    



In [ ]:
import sys
import albumentations
import tensorrt as trt
import torch
import timm
print(torch.__version__)

sys.path.append('/kaggle/input/timm-clone/pytorch-image-models/')
sys.path.append('/kaggle/input/folder')
sys.path.append('/kaggle/input/rsna-src/')

In [ ]:
!pip install /kaggle/input/rsna-src/onnx_graphsurgeon-0.3.25-py2.py3-none-any.whl
!cp -r /kaggle/input/rsna-src/torch2trt/ /tmp/pip/cache/
%cd /tmp/pip/cache/torch2trt
!python setup.py install

In [ ]:
!cp -r /kaggle/input/rsna-src/onnx-tensorrt-main/ /tmp/pip/cache/
%cd /tmp/pip/cache/onnx-tensorrt-main
!python setup.py install
!cp /kaggle/input/easy-load-the-image-with-nvjpeg2000/nvjpeg2k.so ./

In [ ]:
!cp -r /kaggle/input/rsna-src/torch2trt/ /tmp/pip/cache/
%cd /tmp/pip/cache/torch2trt
!python setup.py install

In [ ]:
import pandas as pd
import numpy as np
import cv2

from timeit import default_timer as timer
from joblib import Parallel, delayed
from glob import glob
from tqdm.notebook import tqdm
import os
import sys
import pydicom
import dicomsdl
import nvjpeg2k

###########################################################################################
#from pydicom.pixel_data_handlers.util import apply_voi_lut

from typing import (
    Dict, Optional, Union, List, Tuple, TYPE_CHECKING, cast, Iterable,
    ByteString
)
from pydicom.valuerep import VR

def apply_voi_lut(
    arr: "np.ndarray",
    ds: "Dataset",
    index: int = 0,
    prefer_lut: bool = True
) -> "np.ndarray":
    """Apply a VOI lookup table or windowing operation to `arr`.

    .. versionadded:: 1.4

    .. versionchanged:: 2.1

        Added the `prefer_lut` keyword parameter

    Parameters
    ----------
    arr : numpy.ndarray
        The :class:`~numpy.ndarray` to apply the VOI LUT or windowing operation
        to.
    ds : dataset.Dataset
        A dataset containing a :dcm:`VOI LUT Module<part03/sect_C.11.2.html>`.
        If (0028,3010) *VOI LUT Sequence* is present then returns an array
        of ``np.uint8`` or ``np.uint16``, depending on the 3rd value of
        (0028,3002) *LUT Descriptor*. If (0028,1050) *Window Center* and
        (0028,1051) *Window Width* are present then returns an array of
        ``np.float64``. If neither are present then `arr` will be returned
        unchanged.
    index : int, optional
        When the VOI LUT Module contains multiple alternative views, this is
        the index of the view to return (default ``0``).
    prefer_lut : bool
        When the VOI LUT Module contains both *Window Width*/*Window Center*
        and *VOI LUT Sequence*, if ``True`` (default) then apply the VOI LUT,
        otherwise apply the windowing operation.

    Returns
    -------
    numpy.ndarray
        An array with applied VOI LUT or windowing operation.

    Notes
    -----
    When the dataset requires a modality LUT or rescale operation as part of
    the Modality LUT module then that must be applied before any windowing
    operation.

    See Also
    --------
    :func:`~pydicom.pixel_data_handlers.util.apply_modality_lut`
    :func:`~pydicom.pixel_data_handlers.util.apply_voi`
    :func:`~pydicom.pixel_data_handlers.util.apply_windowing`

    References
    ----------
    * DICOM Standard, Part 3, :dcm:`Annex C.11.2
      <part03/sect_C.11.html#sect_C.11.2>`
    * DICOM Standard, Part 3, :dcm:`Annex C.8.11.3.1.5
      <part03/sect_C.8.11.3.html#sect_C.8.11.3.1.5>`
    * DICOM Standard, Part 4, :dcm:`Annex N.2.1.1
      <part04/sect_N.2.html#sect_N.2.1.1>`
    """
    valid_voi = False
    if ds.get('VOILUTSequence'):
        ds.VOILUTSequence = cast(List["Dataset"], ds.VOILUTSequence)
        valid_voi = None not in [
            ds.VOILUTSequence[0].get('LUTDescriptor', None),
            ds.VOILUTSequence[0].get('LUTData', None)
        ]
    valid_windowing = None not in [
        ds.get('WindowCenter', None),
        ds.get('WindowWidth', None)
    ]

    if valid_voi and valid_windowing:
        if prefer_lut:
            return apply_voi(arr, ds, index)

        return apply_windowing(arr, ds, index)

    if valid_voi:
        return apply_voi(arr, ds, index)

    if valid_windowing:
        return apply_windowing(arr, ds, index)

    return arr


def apply_voi(
    arr: "np.ndarray", ds: "Dataset", index: int = 0
) -> "np.ndarray":
    """Apply a VOI lookup table to `arr`.

    .. versionadded:: 2.1

    Parameters
    ----------
    arr : numpy.ndarray
        The :class:`~numpy.ndarray` to apply the VOI LUT to.
    ds : dataset.Dataset
        A dataset containing a :dcm:`VOI LUT Module<part03/sect_C.11.2.html>`.
        If (0028,3010) *VOI LUT Sequence* is present then returns an array
        of ``np.uint8`` or ``np.uint16``, depending on the 3rd value of
        (0028,3002) *LUT Descriptor*, otherwise `arr` will be returned
        unchanged.
    index : int, optional
        When the VOI LUT Module contains multiple alternative views, this is
        the index of the view to return (default ``0``).

    Returns
    -------
    numpy.ndarray
        An array with applied VOI LUT.

    See Also
    --------
    :func:`~pydicom.pixel_data_handlers.util.apply_modality_lut`
    :func:`~pydicom.pixel_data_handlers.util.apply_windowing`

    References
    ----------
    * DICOM Standard, Part 3, :dcm:`Annex C.11.2
      <part03/sect_C.11.html#sect_C.11.2>`
    * DICOM Standard, Part 3, :dcm:`Annex C.8.11.3.1.5
      <part03/sect_C.8.11.3.html#sect_C.8.11.3.1.5>`
    * DICOM Standard, Part 4, :dcm:`Annex N.2.1.1
      <part04/sect_N.2.html#sect_N.2.1.1>`
    """
    if not ds.get('VOILUTSequence'):
        return arr

    if not np.issubdtype(arr.dtype, np.integer):
        print(#warnings.warn
            "Applying a VOI LUT on a float input array may give "
            "incorrect results"
        )

    # VOI LUT Sequence contains one or more items
    item = cast(List["Dataset"], ds.VOILUTSequence)[index]
    lut_descriptor = cast(List[int], item.LUTDescriptor)
    nr_entries = lut_descriptor[0] or 2**16
    first_map = lut_descriptor[1]

    # PS3.3 C.8.11.3.1.5: may be 8, 10-16
    nominal_depth = lut_descriptor[2]
    if nominal_depth in list(range(10, 17)):
        dtype = 'uint16'
    elif nominal_depth == 8:
        dtype = 'uint8'
    else:
        raise NotImplementedError(
            f"'{nominal_depth}' bits per LUT entry is not supported"
        )

    # Ambiguous VR, US or OW
    unc_data: Iterable[int]
    if item['LUTData'].VR == VR.OW:
        endianness = '<' if ds.is_little_endian else '>'
        unpack_fmt = f'{endianness}{nr_entries}H'
        unc_data = unpack(unpack_fmt, cast(bytes, item.LUTData))
    else:
        unc_data = cast(List[int], item.LUTData)

    lut_data: "np.ndarray" = np.asarray(unc_data, dtype=dtype)

    # IVs < `first_map` get set to first LUT entry (i.e. index 0)
    clipped_iv = np.zeros(arr.shape, dtype=dtype)
    # IVs >= `first_map` are mapped by the VOI LUT
    # `first_map` may be negative, positive or 0
    mapped_pixels = arr >= first_map
    clipped_iv[mapped_pixels] = arr[mapped_pixels] - first_map
    # IVs > number of entries get set to last entry
    np.clip(clipped_iv, 0, nr_entries - 1, out=clipped_iv)

    return cast("np.ndarray", lut_data[clipped_iv])


def apply_windowing(
    arr: "np.ndarray", ds: "Dataset", index: int = 0
) -> "np.ndarray":
    """Apply a windowing operation to `arr`.

    .. versionadded:: 2.1

    Parameters
    ----------
    arr : numpy.ndarray
        The :class:`~numpy.ndarray` to apply the windowing operation to.
    ds : dataset.Dataset
        A dataset containing a :dcm:`VOI LUT Module<part03/sect_C.11.2.html>`.
        If (0028,1050) *Window Center* and (0028,1051) *Window Width* are
        present then returns an array of ``np.float64``, otherwise `arr` will
        be returned unchanged.
    index : int, optional
        When the VOI LUT Module contains multiple alternative views, this is
        the index of the view to return (default ``0``).

    Returns
    -------
    numpy.ndarray
        An array with applied windowing operation.

    Notes
    -----
    When the dataset requires a modality LUT or rescale operation as part of
    the Modality LUT module then that must be applied before any windowing
    operation.

    See Also
    --------
    :func:`~pydicom.pixel_data_handlers.util.apply_modality_lut`
    :func:`~pydicom.pixel_data_handlers.util.apply_voi`

    References
    ----------
    * DICOM Standard, Part 3, :dcm:`Annex C.11.2
      <part03/sect_C.11.html#sect_C.11.2>`
    * DICOM Standard, Part 3, :dcm:`Annex C.8.11.3.1.5
      <part03/sect_C.8.11.3.html#sect_C.8.11.3.1.5>`
    * DICOM Standard, Part 4, :dcm:`Annex N.2.1.1
      <part04/sect_N.2.html#sect_N.2.1.1>`
    """
    if "WindowWidth" not in ds and "WindowCenter" not in ds:
        return arr

    if ds.PhotometricInterpretation not in ['MONOCHROME1', 'MONOCHROME2']:
        raise ValueError(
            "When performing a windowing operation only 'MONOCHROME1' and "
            "'MONOCHROME2' are allowed for (0028,0004) Photometric "
            "Interpretation"
        )

    # May be LINEAR (default), LINEAR_EXACT, SIGMOID or not present, VM 1
    voi_func = cast(str, getattr(ds, 'VOILUTFunction', 'LINEAR')).upper()
    # VR DS, VM 1-n
    elem = ds['WindowCenter']
    center = (
        cast(List[float], elem.value)[index] if elem.VM > 1 else elem.value
    )
    center = cast(float, center)
    elem = ds['WindowWidth']
    width = cast(List[float], elem.value)[index] if elem.VM > 1 else elem.value
    width = cast(float, width)

    # The output range depends on whether or not a modality LUT or rescale
    #   operation has been applied
    ds.BitsStored = cast(int, ds.BitsStored)
    y_min: float
    y_max: float
    if ds.get('ModalityLUTSequence'):
        # Unsigned - see PS3.3 C.11.1.1.1
        y_min = 0
        item = cast(List["Dataset"], ds.ModalityLUTSequence)[0]
        bit_depth = cast(List[int], item.LUTDescriptor)[2]
        y_max = 2**bit_depth - 1
    elif ds.PixelRepresentation == 0:
        # Unsigned
        y_min = 0
        y_max = 2**ds.BitsStored - 1
    else:
        # Signed
        y_min = -2**(ds.BitsStored - 1)
        y_max = 2**(ds.BitsStored - 1) - 1

    slope = ds.get('RescaleSlope', None)
    intercept = ds.get('RescaleIntercept', None)
    if slope is not None and intercept is not None:
        ds.RescaleSlope = cast(float, ds.RescaleSlope)
        ds.RescaleIntercept = cast(float, ds.RescaleIntercept)
        # Otherwise its the actual data range
        y_min = y_min * ds.RescaleSlope + ds.RescaleIntercept
        y_max = y_max * ds.RescaleSlope + ds.RescaleIntercept

    y_range = y_max - y_min
    arr = arr.astype('float32')
    #arr = arr.astype('float64')

    if voi_func in ['LINEAR', 'LINEAR_EXACT']:
        # PS3.3 C.11.2.1.2.1 and C.11.2.1.3.2
        if voi_func == 'LINEAR':
            if width < 1:
                raise ValueError(
                    "The (0028,1051) Window Width must be greater than or "
                    "equal to 1 for a 'LINEAR' windowing operation"
                )
            center -= 0.5
            width -= 1
        elif width <= 0:
            raise ValueError(
                "The (0028,1051) Window Width must be greater than 0 "
                "for a 'LINEAR_EXACT' windowing operation"
            )

        below = arr <= (center - width / 2)
        above = arr > (center + width / 2)
        between = np.logical_and(~below, ~above)

        arr[below] = y_min
        arr[above] = y_max
        if between.any():
            arr[between] = (
                ((arr[between] - center) / width + 0.5) * y_range + y_min
            )
    elif voi_func == 'SIGMOID':
        # PS3.3 C.11.2.1.3.1
        if width <= 0:
            raise ValueError(
                "The (0028,1051) Window Width must be greater than 0 "
                "for a 'SIGMOID' windowing operation"
            )

        arr = y_range / (1 + np.exp(-4 * (arr - center) / width)) + y_min
    else:
        raise ValueError(
            f"Unsupported (0028,1056) VOI LUT Function value '{voi_func}'"
        )

    return arr
###########################################################################################

def time_to_str(t, mode='min'):
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)

    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)

    else:
        raise NotImplementedError


###########################################################################################
def read_image(df, image_dir):
    image = []
    for t,d in df.iterrows():
        image_file = f'{image_dir}/{d.machine_id}/{d.patient_id}/{d.image_id}.png'
        m = cv2.imread(image_file,cv2.IMREAD_ANYDEPTH)
        image.append(m)
    return image

def make_transfer_syntax_uid(df, dcm_dir):
    machine_id_to_transfer = {}
    machine_id = df.machine_id.unique()
    for i in machine_id:
        d = df[df.machine_id == i].iloc[0]
        f = f'{dcm_dir}/{d.patient_id}/{d.image_id}.dcm'
        dicom = pydicom.dcmread(f)
        machine_id_to_transfer[i] = dicom.file_meta.TransferSyntaxUID
    return machine_id_to_transfer

def normalised_to_8bit(image, photometric_interpretation):
    xmin = image.min()
    xmax = image.max()

    norm = np.empty_like(image, dtype=np.uint8)
    dicomsdl.util.convert_to_uint8(image, norm, xmin, xmax)
    if photometric_interpretation == 'MONOCHROME1':
        norm = 255 - norm
    return norm

def resize_image_to_height(image, image_height):
    h, w = image.shape[:2]
    s = image_height/h
    if image_height!=h:
        image = cv2.resize(image, dsize=None, fx=s, fy=s, interpolation=cv2.INTER_LINEAR)
    return image

#----------------------------------------------------------------
# dicomsdl reader
def dicomsdl_to_numpy_image(ds, index=0):
    # https://stackoverflow.com/questions/44659924/returning-numpy-arrays-via-pybind11
    info = ds.getPixelDataInfo()
    if info['SamplesPerPixel'] != 1:
        raise RuntimeError('SamplesPerPixel != 1')

    shape = [info['Rows'], info['Cols']]
    dtype = info['dtype']
    outarr = np.empty(shape, dtype=dtype)
    ds.copyFrameData(index, outarr)
    return outarr

def dicomsdl_parallel_process(d, dcm_dir, image_dir, image_height, is_voi_lut):
    dcm_file = f'{dcm_dir}/{d.patient_id}/{d.image_id}.dcm'
    ds = dicomsdl.open(dcm_file)
    image = dicomsdl_to_numpy_image(ds)
    image = resize_image_to_height(image, image_height)

    if is_voi_lut:
        dc = pydicom.dcmread(dcm_file)
        image = apply_voi_lut(image, dc)
        image = image.astype(np.float32)
    image = normalised_to_8bit(image, ds.PhotometricInterpretation)  # +1

    # save as png
    os.makedirs(f'{image_dir}/', exist_ok=True)
    cv2.imwrite(f'{image_dir}/{d.patient_id}_{d.image_id}.png', image)

def process_non_j2k(df, dcm_dir, image_dir, image_height, n_jobs, is_voi_lut=True):
    #https://stackoverflow.com/questions/56659294/does-joblib-parallel-keep-the-original-order-of-data-passed
    #Parallel(n_jobs=2, backend='multiprocessing')(
    Parallel(n_jobs=n_jobs)(
        delayed(dicomsdl_parallel_process)(d, dcm_dir, image_dir, image_height, is_voi_lut)
        for t,d in tqdm(df.iterrows())
    )

#----------------------------------------------------------------
# nvjpeg2k reader

'''
TransferSyntaxUID
1.2.840.10008.1.2.4.70 = JPEG Lossless, Nonhierarchical, First- Order Prediction (Processes 14)
1.2.840.10008.1.2.4.90 = JPEG 2000 Image Compression (Lossless Only)
'''
j2k_decoder = nvjpeg2k.Decoder()

def process_j2k(df, dcm_dir, image_dir, image_height, is_voi_lut=True):
    for t, d in tqdm(df.iterrows()):
        dcm_file = f'{dcm_dir}/{d.patient_id}/{d.image_id}.dcm'
        dc = pydicom.dcmread(dcm_file)
        offset = dc.PixelData.find(b'\x00\x00\x00\x0C')
        jpeg_stream = bytearray(dc.PixelData[offset:])
        image = j2k_decoder.decode(jpeg_stream)
        image = resize_image_to_height(image, image_height)
        #     jpeg_stream = bytearray(ds.PixelData[offset:])
        #     jpeg_stream = np.array(bytearray(ds.PixelData[offset:]),np.uint8)

        if is_voi_lut:
            image = apply_voi_lut(image, dc)
            image = image.astype(np.float32)
        image = normalised_to_8bit(image, dc.PhotometricInterpretation)

        # save as png
        os.makedirs(f'{image_dir}/', exist_ok=True)
        cv2.imwrite(f'{image_dir}/{d.patient_id}_{d.image_id}.png', image)

In [ ]:

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda.amp as amp
import torch2trt
print( 'torch.cuda.device_count() = %d'%torch.cuda.device_count())
print( 'torch.cuda.get_device_properties() = %s' % str(torch.cuda.get_device_properties(0))[21:])

import timm
print('timm',timm.__version__)

import nvjpeg2k
print('import ok!')

In [ ]:
mode = [
   'do-dicom-to-png',
   'do-breast-box'
]
DEBUG = False

convert_height = 2560
image_height = 2048
image_width  = 1280

if DEBUG == True :
    csv_file = '/kaggle/input/rsna-breast-cancer-detection/train.csv'
    dcm_dir  = '/kaggle/input/rsna-breast-cancer-detection/train_images'
    test_df = pd.read_csv(csv_file)[:503]
else:
    csv_file = '/kaggle/input/rsna-breast-cancer-detection/test.csv'
    dcm_dir  = '/kaggle/input/rsna-breast-cancer-detection/test_images'
    test_df = pd.read_csv(csv_file)[:]

machine_id_to_transfer = make_transfer_syntax_uid(test_df, dcm_dir)
test_df.loc[:, 'i'] = np.arange(len(test_df))
test_df.loc[:, 'TransferSyntaxUID'] = test_df.machine_id.map(machine_id_to_transfer)

print('test_df', test_df.shape)
print(test_df)
print('')

In [ ]:
import torch.onnx
import onnx
import gc
from torch2trt import TRTModule

In [ ]:
checkpoint = [
            '/kaggle/input/rsna-fin/convnext_large_1536_0_3.pth',
            '/kaggle/input/rsna-fin/b4_fold0_2_pos1_c4.pth',
            '/kaggle/input/rsna-fin/cvs_fold0_3_pos1_c4.pth',
            '/kaggle/input/rsna-fin/v2s_fold0_3_pos1_c4.pth',
]
models = []
for i in range(len(checkpoint)):
    model_trt = TRTModule()
    model_trt.load_state_dict(torch.load(checkpoint[i]))
    models.append(model_trt)
    
gc.collect()
torch.cuda.empty_cache()

In [ ]:
png_dir = '/kaggle/tmp/~png' 

j2k_df = test_df[test_df.TransferSyntaxUID == '1.2.840.10008.1.2.4.90'].reset_index(drop=True)
non_j2k_df = test_df[test_df.TransferSyntaxUID != '1.2.840.10008.1.2.4.90'].reset_index(drop=True)

print(f'process_j2k(): {len(j2k_df)}')
start_timer = timer()
process_j2k(j2k_df, dcm_dir, png_dir, convert_height, is_voi_lut= True)
print(time_to_str(timer() - start_timer, 'sec'))

print(f'process_non_j2k(): {len(non_j2k_df)}')
start_timer = timer()
process_non_j2k(non_j2k_df, dcm_dir, png_dir, convert_height, n_jobs=2, is_voi_lut= True)  
print(time_to_str(timer() - start_timer, 'sec'))

In [ ]:
class bboxDataset(torch.utils.data.Dataset):
    def __init__(self, df,transform = None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        d = self.df.iloc[i]
        patient_id = self.df['patient_id'].iloc[i]
        image_id = self.df['image_id'].iloc[i]


        image = np.zeros((256, 256), np.uint8)

        img = cv2.imread(f'/kaggle/tmp/~png/{patient_id}_{image_id}.png', cv2.IMREAD_GRAYSCALE)    
                
        h, w = img.shape

        s = 256 / h
        h, w = int(s * h), int(s * w)
        m = cv2.resize(img, dsize=(w, h), interpolation=cv2.INTER_LINEAR)
        y = (256 - h) // 2
        x = (256 - w) // 2
        rect = (x, y, x + w, y + h)

        image[y:y + h, x:x + w] = m
        image = image.astype(np.float32) / 255

        r = {}
        r['index'] = i
        r['d'    ] = d
        r['rect' ] = rect
        r['image'] = torch.from_numpy(image).float()
        return r

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

from timm.models.resnet import resnet34d

import cv2
import numpy as np


##############################################################
class PreprocessNet(nn.Module):

    def __init__(self,):
        super(PreprocessNet, self).__init__()
        self.register_buffer('mean', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))
        self.register_buffer('std', torch.FloatTensor([0.5, 0.5, 0.5]).reshape(1, 3, 1, 1))

        self.encoder = resnet34d(pretrained=False)
        self.encoder.maxpool = nn.Identity()
        dim = 512

        self.mask = nn.Sequential(
            nn.Conv2d(dim,4*16, kernel_size = 3, padding=1),
            nn.PixelShuffle(2),
            nn.ReLU(True),
            nn.Conv2d(16,4, kernel_size = 3, padding=1),
            nn.PixelShuffle(2),
        )
        self.laterality = nn.Linear(dim,1)


    def forward(self, batch):
        x = batch['image']
        x = (x - self.mean) / self.std
        e = self.encoder.forward_features(x)
        mask = self.mask(e)
        mask = torch.sigmoid(mask)
        mask  = F.interpolate(mask,size=x.shape[-2:],mode='bilinear',align_corners=False)

        pool = F.adaptive_avg_pool2d(e,1)
        pool = torch.flatten(pool,1)
        laterality = self.laterality(pool).reshape(-1)
        laterality = torch.sigmoid(laterality)

        output={
            'mask':mask,
            'laterality':laterality,
        }
        return output

##############################################################

def proprocess_collate(batch):
    d = {}
    key = batch[0].keys()
    for k in key:
        v = [b[k] for b in batch]
        d[k] = v
    d['image'] = torch.stack(d['image'],0).unsqueeze(1)
    return d

#-----------------------------------------

def predict_to_mask(predict):

    #for image size 224
    erosion_size = 7
    min_diff = 224

    #----
    thresh = ((predict>0.5)*255).astype(np.uint8)
    contour, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # CHAIN_APPROX_NONE     CHAIN_APPROX_SIMPLE
    c = max(contour, key=cv2.contourArea)
    m0 = np.zeros(shape=thresh.shape, dtype=np.uint8)
    cv2.drawContours(m0, [c], -1, 255, thickness=-1)

    element = cv2.getStructuringElement( cv2.MORPH_ELLIPSE, (2 * erosion_size + 1, 2 * erosion_size + 1), (erosion_size, erosion_size))
    m1 = cv2.erode(m0, element)
    contour, hierarchy = cv2.findContours(m1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  # CHAIN_APPROX_NONE     CHAIN_APPROX_SIMPLE
    c = max(contour, key=cv2.contourArea)
    m1[...]=0
    cv2.drawContours(m1, [c], -1, 255, thickness=-1)

    m2 = cv2.dilate(m1, element)

    ##----------
    ss = (m0 == m2).sum()
    if ss < min_diff:
        mask = m0
    else:
        mask = m2

    mask = mask/255
    return mask

def mask_to_box(mask, pad=0.03):
    yy, xx = np.where(mask > 0)
    xmin = xx.min()
    xmax = xx.max()
    ymin = yy.min()
    ymax = yy.max()

    h,w = mask.shape
    pad = int(pad*h)
    xmin = max(0, xmin - pad)
    xmax = min(w, xmax + pad)
    ymin = max(0, ymin - pad)
    ymax = min(h, ymax + pad)
    return xmin, ymin, xmax, ymax


def post_process(batch, output):
    rect  = batch['rect']
    image = [m for m in batch['image'].float().data.cpu().numpy().squeeze(1)]
    mask  = [p for p in output['mask'].float().data.cpu().numpy().squeeze(1)]
    laterality = (output['laterality'].float().data.cpu().numpy() > 0.5).astype(int).tolist()
    box = []

    batch_size = len(batch['index'])
    for b in range(batch_size):
        x0, y0, x1, y1 = rect[b]
        m = image[b][y0:y1, x0:x1]
        p = mask[b][y0:y1, x0:x1]

        h = y1-y0
        p = predict_to_mask(p)
        xmin, ymin, xmax, ymax = mask_to_box(p)
        box.append((xmin/h, ymin/h, xmax/h, ymax/h))
        image[b] = m
        mask[b] = p

    output = {
        'image' : image,
        'mask' : mask,
        'laterality' : laterality,
        'box' : box,
    }
    return output


#-----------------------------------------
def draw_shadow_text(img, text, pt,  fontScale, color, thickness, color1=None, thickness1=None):
    if color1 is None: color1=(0,0,0)
    if thickness1 is None: thickness1 = thickness+2
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, text, pt, font, fontScale, color1, thickness1, cv2.LINE_AA)
    cv2.putText(img, text, pt, font, fontScale, color,  thickness,  cv2.LINE_AA)


def image_show(name, image, type='bgr', resize=1):
    if type == 'rgb': image = np.ascontiguousarray(image[:, :, ::-1])
    H, W = image.shape[0:2]

    cv2.namedWindow(name, cv2.WINDOW_GUI_NORMAL)  # WINDOW_NORMAL
    # cv2.namedWindow(name, cv2.WINDOW_GUI_EXPANDED)  #WINDOW_GUI_EXPANDED
    cv2.imshow(name, image)  # .astype(np.uint8))
    cv2.resizeWindow(name, round(resize * W), round(resize * H))



def draw_preprocess_overlay(image, mask, box, laterality):
    h,w = image.shape
    xmin, ymin, xmax, ymax = box
    xmin = int(xmin*h)
    ymin = int(ymin*h)
    xmax = int(xmax*h)
    ymax = int(ymax*h)

    overlay = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    overlay = 1 - (1 - mask[..., np.newaxis] * [[[1, 0, 0]]]) * (1 - overlay)
    cv2.rectangle(overlay, (xmin, ymin), (xmax, ymax), (0, 0, 1), 2)

    if laterality == 0:
        draw_shadow_text(overlay, 'left', (5, 20), 0.7, (1, 1, 1), 2)
    if laterality == 1:
        draw_shadow_text(overlay, 'right', (overlay.shape[1] - 55, 20), 0.7, (1, 1, 1), 2)
    return overlay


In [ ]:
dataset = bboxDataset(test_df)
bboxloader = DataLoader(
    dataset,
    sampler = SequentialSampler(dataset),
    batch_size  = 64,
    drop_last   = False,
    num_workers = 2,
    pin_memory  = False,
    collate_fn = proprocess_collate,
)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
checkpoint = f'/kaggle/input/crop-breast-region-etc/00000387.model.pth'  

net = PreprocessNet()
f = torch.load(checkpoint, map_location=lambda storage, loc: storage)
net.load_state_dict(f['state_dict'],strict=True)
net.cuda()
net.eval()
box=[]
for t, batch in tqdm(enumerate(bboxloader), total=len(bboxloader)):
    batch_size = len(batch['index'])
    for k in ['image']: batch[k] = batch[k].cuda()

    with torch.no_grad():
        with amp.autocast(enabled = True):
            output = net(batch)
    
    output = post_process(batch, output)
    for b in range(batch_size):
        box.append(output['box'][b])


In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
bbox = pd.DataFrame(np.asarray(box).tolist())
test_df['xmin']= bbox[0]
test_df['ymin']= bbox[1]
test_df['xmax']= bbox[2]
test_df['ymax']= bbox[3]

In [ ]:
if DEBUG == True :
    all_images=glob('/kaggle/tmp/~png/*.png')
    import random
    idx = random.randint(0, 3)
    plt.imshow(cv2.imread(all_images[idx]))

In [ ]:
class rDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        patient_id = self.df['patient_id'].iloc[i]
        image_id = self.df['image_id'].iloc[i]

        
        img = cv2.imread(f'/kaggle/tmp/~png/{patient_id}_{image_id}.png',cv2.IMREAD_GRAYSCALE)
        h, w = img.shape
        xmin, ymin, xmax, ymax = self.df['xmin'].iloc[i],self.df['ymin'].iloc[i], self.df['xmax'].iloc[i], self.df['ymax'].iloc[i] 
        xmin = int(xmin*h)
        ymin = int(ymin*h)
        xmax = int(xmax*h)
        ymax = int(ymax*h)
        img = img[ymin:ymax,xmin:xmax]
        
        img = np.stack([(img)], axis=2)
        
        if self.transform is not None:
            img = self.transform(image=img)['image']
        #cv2.imwrite(f'/kaggle/working/{i}.png', img)
        if DEBUG == True :
            cv2.imwrite(f'/kaggle/working/{i}.png', img)
        img = img/255.
        img = img.transpose(2,0,1)
        img = torch.tensor(img).float()
        
        return img

In [ ]:
test_transform = albumentations.Compose([
    albumentations.LongestMaxSize(max_size=image_height,p=1),
    albumentations.PadIfNeeded(min_width=image_width,p=1),
    albumentations.Resize(height=image_height, width=image_width,p=1),
])

In [ ]:
batch_size = 1
test_set = rDataset(df = test_df, transform=test_transform)
test_loader = DataLoader(test_set, shuffle=False, batch_size=batch_size, num_workers=2, pin_memory=True)


In [ ]:
result = {
    'probability': [],
}
TTA = False
start_timer = timer()
for idx, (img) in tqdm(enumerate(test_loader), total=len(test_loader)):
    image0 = img.half().cuda()
#     image1 = torch.flip(image0, dims=[3, ])  # TTA
#     image2 = torch.flip(image0, dims=[1,2])
    p = 0
    count = 0
    with torch.no_grad():
        with amp.autocast(enabled=True):
            for i in range(len(models)):
                a = torch.sigmoid(models[i](image0))
                p += a
                count += 1
#                 if(TTA==True):
#                     c = a.detach().cpu().numpy()
#                     if((np.max(c) > 0.7)):
#                         a = torch.sigmoid(models[i](image1))
#                         p += a
#                         count += 1

#                         a = torch.sigmoid(models[i](image2))
#                         p += a
#                         count += 1

    p = p / count
    result['probability'].append(p.float().detach().cpu().numpy())
print('')

probability = np.concatenate(result['probability']).flatten()
np.save('probability.npy', probability)

####################################################
pred = np.load('probability.npy')
print('')
pred_df = pd.DataFrame({'prediction_id': test_df.prediction_id, 'cancer': pred})
sub = pred_df.groupby('prediction_id')[['cancer']].agg('mean')
th = np.quantile(sub['cancer'].values, 0.97935)
sub['cancer'] = (sub['cancer'].values > th).astype(int)

sub.to_csv('/kaggle/working/submission.csv', index=True)
#print('submit_df', submit_df)